# Simulation with data from imu
#### the codes of data obtaining, modeling and simulation functions are already wroten in other py files under the same folder

In [2]:
# Files
from get_data import *
from model import *
from model_kf import *
# from model_3d import *

# Packages
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    p.title.text_font_size = "15pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

Data of 'imu':  ['time', 'w_x', 'w_y', 'w_z', 'a_x', 'a_y', 'a_z']
Data of 'tf':  ['time', 'r_w', 'r_x', 'r_y', 'r_z', 'x', 'y', 'z']
Data of velocity 'vel':  ['time', 'velocity']


Loading BokehJS ...

## A simple planar model of Hamster:
$$ 
X = \begin{bmatrix}
x \\ 
y \\ 
\theta \\ 
v 
\end{bmatrix}, \:
u = \begin{bmatrix}
a_z \\
\omega_y
\end{bmatrix}
$$

$$
\dot{X}= 
\begin{bmatrix}
v\cdot \cos\theta \\
v\cdot \sin\theta \\
-\omega_y \\ 
a_z
\end{bmatrix} = 
\begin{bmatrix}
0 & 0 & 0 & \cos\theta\\ 
0 & 0 & 0 & \sin\theta\\ 
0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0
\end{bmatrix}
\cdot X +
\begin{bmatrix}
0 & 0 \\ 
0 & 0 \\ 
0 & -1 \\ 
1 & 0 
\end{bmatrix}
\cdot u
$$

In [ ]:
# a = np.concatenate((imu['a_x'], imu['a_y'], imu['a_z']), axis=0)
# w = np.concatenate((imu['w_x'], imu['w_y'], imu['w_z']), axis=0)

# gx = np.average(imu['a_x'][0][0:250])
# gy = np.average(imu['a_y'][0][0:250])
# gz = np.average(imu['a_z'][0][0:250])
# g = np.array([[gx],[gy],[gz]])

# a0 = np.dot(o_R_e, a - g) # eliminate gravity part and correct the rotation 
# w0 = np.dot(o_R_e, w)
# print(a)
# print(a0)



#==========================================Simulation 0================================================
# original data
a_z = imu['a_z']  - 0.33 # compensate the data offset of linear acceleration
w_y = imu['w_y']
U_k = np.append(a_z,w_y,axis=0) # 'input' of simulation system

# # corrected data
# a_z = a0[2].reshape(1,-1)
# w_y = w0[1].reshape(1,-1)
# U_k = np.append(a_z,w_y,axis=0) # 'input' of simulation system


timestamp = imu['time'] - imu['time'][0][0] # time start from 0s
Ts = np.diff(timestamp)  # time steps of time stamps, number of data will be -1


x0 = [0,0,0,0]  # initial states
x_all = sim(x0, U_k, timestamp)
print(x_all.shape)

In [ ]:
# ============================================ Plot =============================================
p = plot_config(width=600, height=600, title='imu curve',  x_label='x [ unknown ]', y_label= 'y [ unknown ]')
p.circle(x_all[0][0], x_all[1][0], fill_color="red",legend_label='start', size=10)
p.line(x=x_all[0], y=x_all[1],legend_label='curve', line_width=1, line_color='black')
p.asterisk(x_all[0][-1], x_all[1][-1], line_color="green",legend_label='end', size=10)

show(p, notebook_handle=True)

p2 = plot_config(width=600, height=600, title='tf translation curve',  x_label='x [ unknown ]', y_label= 'y [ unknown ]')
p2.circle(tf['x'][0][0], tf['y'][0][0], fill_color="red",legend_label='start', size=10)
p2.line(x=tf['x'][0], y=tf['y'][0],legend_label='curve', line_width=1, line_color='black')
p2.asterisk(tf['x'][0][-1], tf['y'][0][-1], line_color="green",legend_label='end', size=10)

show(p2, notebook_handle=True)


p1 = plot_config(width=900, height=600, title='system states',  x_label='t [ s ]', y_label= 'states [ unknown ] ')
p1.line(x=timestamp[0], y=x_all[0],legend_label='x', line_width=2, line_color='green')
p1.line(x=timestamp[0], y=x_all[1],legend_label='y', line_width=2, line_color='black')
p1.line(x=timestamp[0], y=x_all[2],legend_label='theta', line_width=2, line_color='royalblue')
p1.line(x=timestamp[0], y=x_all[3],legend_label='v', line_width=2, line_color='crimson')

show(p1, notebook_handle=True)

## 3D model of Hamster:
$$ 
\tilde{X} = \begin{bmatrix}
^{0}r_c \\ 
^{0}v_c \\ 
^{0}R_c
\end{bmatrix}, \;
u = 
\begin{bmatrix}
^{c}\omega_c \\
^{c}a_c 
\end{bmatrix} = 
\begin{bmatrix}
\omega_x \\
\omega_y \\
\omega_z \\
a_x \\
a_y \\
a_z 
\end{bmatrix}, \; s.t.
\left\{\; \begin{matrix}
^{0}r_c : vector\,(3\times1)\\
^{0}v_c : vector\,(3\times1)\\
^{0}R_c : matrix\,(3\times3)
\end{matrix}\right.
\;totally\;15\;states,\;\;
\tilde{X} \;\Rightarrow \; X\;(15\times1)
$$

$$
\dot{\tilde{X}}= 
\begin{bmatrix}
^{0}v_c \\
^{0}a_c \\
^{0}\dot{R}_c
\end{bmatrix} = 
\begin{bmatrix}
^{0}v_c \\
^{0}R_c \cdot\, ^{c}a_c \\
B(^{0}\omega_c) \cdot \,^{0}R_c
\end{bmatrix} =
\begin{bmatrix}
^{0}v_c \\
^{0}R_c \cdot\, ^{c}a_c \\
^{0}R_c \cdot B(^{c}\omega_c)
\end{bmatrix},\;
s.t.\;B(\omega) = 
\begin{bmatrix}
0       &   -\omega_z   & \omega_y\\ 
\omega_z   &   0        & -\omega_x\\ 
-\omega_y   &   \omega_x   & 0
\end{bmatrix}
$$

$$
\begin{aligned}
^{0}\dot{R}_c &= B(^{0}\omega_c) \cdot \,^{0}R_c \\
&= B(^{0}R_c \cdot\, ^{c}\omega_c) \cdot \,^{0}R_c \\
&=\,^{0}R_c \cdot B(^{c}\omega_c) \cdot \,^{0}R_c^T \cdot \,^{0}R_c\\
&=\,^{0}R_c \cdot B(^{c}\omega_c)
\end{aligned}
$$

In [ ]:
pi = np.pi
# ==========================================Simulation================================================
U_k = np.concatenate((imu['w_x'], imu['w_y'], imu['w_z'],
                      imu['a_x'], imu['a_y'], imu['a_z']),
                     axis=0)  # 'input' of simulation system

timestamp = imu['time'] - imu['time'][0][0]  # time start from 0s
Ts = np.diff(timestamp)  # time steps of time stamps, number of data will be minus 1

r0 = np.array([[0], [0], [0]])
v0 = np.array([[0], [0], [0]])
R0 = 2 * np.array([[ 0,  0, 1],
                   [-1,  0, 0],
                   [ 0, -1, 0]])

x0 = reshape_m2x(r0, v0, R0)  # initial states

x_all = sim_3d(x0, U_k, Ts)
print(x_all.shape)

In [ ]:
# ============================================ Plot =============================================
r = x_all[0:3,:]
v = x_all[3:6,:]
# ------------------------------3d----------------------------
%matplotlib notebook
# Create a 3D figure
fig = plt.figure()
ax = plt.axes(projection="3d")
# # Set the x, y, and z limits
# ax.set_xlim([-1, 1])
# ax.set_ylim([-1, 1])
# ax.set_zlim([-1, 1])
# Plot the data points
ax.scatter3D(r[0], r[1], r[2],s=2,c="k")

# Set the axis labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('3D Point Diagram')
# Show the plot
plt.show()

p1 = plot_config(width=900, height=600, title='velocity',  x_label='t [ s ]', y_label= 'velocity [ m/s ] ')
p1.line(x=timestamp[0], y=v[0],legend_label='x', line_width=2, line_color='black')
p1.line(x=timestamp[0], y=v[1],legend_label='y', line_width=2, line_color='royalblue')
p1.line(x=timestamp[0], y=v[2],legend_label='z', line_width=2, line_color='crimson')
show(p1, notebook_handle=True)